<a href="https://colab.research.google.com/github/MarioSigal/Aprendizaje-Automatico-I-y-II/blob/main/TP_2_Aprendizaje_Automatico_0_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Imports


In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel
from typing import List, Dict, Any
import numpy as np
import pandas as pd
from google.colab import drive

#DATA FRAME


In [2]:
#Cargar drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!ls /content/drive/MyDrive/

 colab
'Colab Notebooks'
'Comprobante de inscripcion a Analisis Avanzado (Ma y Ju 18 a 22).pdf'
 cosas
 costos.xlsx
'cuentas (1).gsheet'
'cuentas (2).gsheet'
 cuentas.gsheet
 cuentas.xlsx
 DataSetsNature
 fotos
'Hoja de cálculo sin título (10).gsheet'
'Hoja de cálculo sin título (11).gsheet'
'Hoja de cálculo sin título (12).gsheet'
'Hoja de cálculo sin título (13).gsheet'
'Hoja de cálculo sin título (1).gsheet'
'Hoja de cálculo sin título (2).gsheet'
'Hoja de cálculo sin título (3).gsheet'
'Hoja de cálculo sin título (4).gsheet'
'Hoja de cálculo sin título (5).gsheet'
'Hoja de cálculo sin título (6).gsheet'
'Hoja de cálculo sin título (7).gsheet'
'Hoja de cálculo sin título (8).gsheet'
'Hoja de cálculo sin título (9).gsheet'
'Hoja de cálculo sin título.gsheet'
 hoteles_filtrados.gsheet
 Laboratorio4_BOWs_ERV2025.ipynb
 modelo_datasetV1_30e_2800p.pt
'Presentación de prototipos.gslides'
'Presentación de prototipos.pptx'
'Presentación sin título.gslides'
'P

In [4]:
df = pd.read_csv("/content/drive/MyDrive/colab/dataset_datos_total2.csv")

In [5]:
# Agrupar tokens por párrafo
df_por_parrafos = df.groupby("instancia_id").agg({
    "token_id":list,
    "token": list,
    "punt_inicial": list,
    "punt_final": list,
    "capitalización": list
}).reset_index()


In [7]:
print(df_por_parrafos["token"][1])

['el', 'nacimiento', 'de', 'un', 'lucha', '##dor', 'por', 'la', 'libertad', 'capítulo', '11', 'capítulo', '12', 'capítulo', '13', 'capítulo', '14', 'parte', 'cuarta', 'la', 'lucha', 'es', 'mi', 'vida', 'capítulo', '15', 'capítulo', '16', 'capítulo', '17', 'capítulo', '18', 'capítulo', '19', 'capítulo', '20', 'capítulo', '21', 'capítulo', '22', 'parte', 'quinta', 'trai', '##ción', 'capítulo', '23', 'capítulo', '24', 'capítulo', '25', 'capítulo', '26', 'capítulo', '27', 'capítulo', '28', 'capítulo', '29', 'capítulo', '30', 'capítulo', '31', 'capítulo', '32', 'capítulo', '33', 'capítulo', '34', 'capítulo', '35', 'capítulo', '36', 'capítulo', '37', 'capítulo', '38', 'capítulo', '39', 'parte', 'sexta']


In [ ]:
print(df_por_parrafos["token"][1])
print(df.shape)

['el', 'nacimiento', 'de', 'un', 'lucha', '##dor', 'por', 'la', 'libertad', 'capítulo', '11', 'capítulo', '12', 'capítulo', '13', 'capítulo', '14', 'parte', 'cuarta', 'la', 'lucha', 'es', 'mi', 'vida', 'capítulo', '15', 'capítulo', '16', 'capítulo', '17', 'capítulo', '18', 'capítulo', '19', 'capítulo', '20', 'capítulo', '21', 'capítulo', '22', 'parte', 'quinta', 'trai', '##ción', 'capítulo', '23', 'capítulo', '24', 'capítulo', '25', 'capítulo', '26', 'capítulo', '27', 'capítulo', '28', 'capítulo', '29', 'capítulo', '30', 'capítulo', '31', 'capítulo', '32', 'capítulo', '33', 'capítulo', '34', 'capítulo', '35', 'capítulo', '36', 'capítulo', '37', 'capítulo', '38', 'capítulo', '39', 'parte', 'sexta']
(10212449, 6)


##TOKENIZER & BERT EMBEDDINGS

In [8]:
"""##TOKENIZER & BERT EMBEDDINGS"""

tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")
model_bert = BertModel.from_pretrained("bert-base-multilingual-cased")
model_bert.eval()


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(119547, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=Fals

In [9]:
# Congelar parámetros de BERT para ahorrar memoria
for param in model_bert.parameters():
    param.requires_grad = False

##CREAR CLASE DATASET PARA PYTORCH

In [25]:
from torch.utils.data import Dataset

class DynamicEmbeddingDataset(Dataset):
    """Dataset de pytorch que calcula embeddings dinámicamente"""
    def __init__(self, df, tokenizer, bert_model):
        self.df = df
        self.tokenizer = tokenizer
        self.bert_model = bert_model
        self.embedding_matrix = bert_model.embeddings.word_embeddings.weight
        self.embedding_matrix.requires_grad = False

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        token_id_list = row["token_id"]

        # Calcular embeddings dinámicamente
        token_embeddings = []
        for token_id in token_id_list:
            if token_id is None or token_id == self.tokenizer.unk_token_id:
                token_id = self.tokenizer.unk_token_id
            emb = self.embedding_matrix[token_id].detach()
            token_embeddings.append(emb)

        # Convertir a tensor
        if token_embeddings:
            embeddings = torch.stack(token_embeddings)
        else:
            embeddings = torch.empty(0, self.embedding_matrix.shape[1])

        # Preparar labels
        labels = {
            "punt_inicial": torch.tensor(row["punt_inicial"], dtype=torch.long),
            "punt_final": torch.tensor(row["punt_final"], dtype=torch.long),
            "capitalización": torch.tensor(row["capitalización"], dtype=torch.long)
        }

        return embeddings, labels

##PADDING

In [26]:
from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch):
    """
    batch: lista de tuplas (embeddings, labels)
    """
    embeddings_list, labels_list = zip(*batch)

    # Pad embeddings (seq_len, embedding_dim) -> (batch_size, max_seq_len, embedding_dim)
    embeddings_padded = pad_sequence(embeddings_list, batch_first=True, padding_value=0.0)

    # Pad labels
    punt_inicial = pad_sequence([l["punt_inicial"] for l in labels_list], batch_first=True, padding_value=-100).long()
    punt_final = pad_sequence([l["punt_final"] for l in labels_list], batch_first=True, padding_value=-100).long()
    capitalizacion = pad_sequence([l["capitalización"] for l in labels_list], batch_first=True, padding_value=-100).long()

    return embeddings_padded, {
        "punt_inicial": punt_inicial,
        "punt_final": punt_final,
        "capitalizacion": capitalizacion
    }

## DATA LOADER

In [33]:
#TEST
#print(int(len(df_por_parrafos)*0.9)) # da 122454, despues agarrar tail de eso
df_por_parrafos_test = df_por_parrafos.head(10000)

from torch.utils.data import DataLoader

# Usar el nuevo dataset con embeddings dinámicos
dataset = DynamicEmbeddingDataset(df_por_parrafos_test, tokenizer, model_bert)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True, collate_fn=collate_fn)


##RNN Unidireccional

####Encoder

In [14]:
class EncoderUnidireccional(nn.Module):
    def __init__(self, embedding_dim=768, hidden_dim=256, num_layers=2, dropout=0.3):
        super(EncoderUnidireccional, self).__init__()
        self.lstm = nn.LSTM(
            input_size=embedding_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout,
            bidirectional=False  # unidireccional
        )

    def forward(self, embeddings):
        """
        embeddings: tensor de forma (batch_size, seq_len, embedding_dim)
        """
        outputs, (hidden, cell) = self.lstm(embeddings)
        # outputs: (batch_size, seq_len, hidden_dim)
        # hidden: (num_layers, batch_size, hidden_dim)
        # cell:   (num_layers, batch_size, hidden_dim)
        return outputs, (hidden, cell)

####Decoder

In [17]:
class DecoderUnidireccional(nn.Module):
    def __init__(self, hidden_dim=256, num_layers=2, dropout=0.3):
        super(DecoderUnidireccional, self).__init__()
        self.lstm = nn.LSTM(
            input_size=hidden_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout,
            bidirectional=False  # unidireccional
        )

        # Capa feed-forward para cada problema
        self.punt_inicial_ff = nn.Linear(hidden_dim, 2)
        self.punt_final_ff = nn.Linear(hidden_dim, 4)
        self.capital_ff = nn.Linear(hidden_dim, 4)

        # Función de activación para cada problema
        #self.punt_inicial_sigmoid = nn.Sigmoid()
        #self.punt_final_softmax = nn.Softmax(dim=4)
        #self.capital_softmax = nn.Softmax(dim=4)


    def forward(self, encoder_outputs, hidden, cell):
        """
        encoder_outputs: (batch_size, seq_len, hidden_dim)
        hidden, cell: del encoder
        """
        outputs, _ = self.lstm(encoder_outputs, (hidden, cell))

        #punt_inicial_logits = self.punt_inicial_sigmoid(self.punt_inicial_ff(outputs))
        #punt_final_logits = self.punt_final_sofmax(self.punt_final_ff(outputs))
        #capital_logits = self.capital_sofmax(self.capital_ff(outputs))

        punt_inicial_logits = self.punt_inicial_ff(outputs)
        punt_final_logits = self.punt_final_ff(outputs)
        capital_logits = self.capital_ff(outputs)


        return {
            "puntuación inicial": punt_inicial_logits,
            "puntuación final": punt_final_logits,
            "capitalización": capital_logits,
        }


####Encoder–Decoder

In [18]:
class ModeloUnidireccional(nn.Module):
    def __init__(self, embedding_dim=768, hidden_dim=256, num_layers=2, dropout=0.3):
        super(ModeloUnidireccional, self).__init__()
        self.encoder = EncoderUnidireccional(embedding_dim, hidden_dim, num_layers, dropout)
        self.decoder = DecoderUnidireccional(hidden_dim, num_layers, dropout)

    def forward(self, embeddings):
        encoder_outputs, (hidden, cell) = self.encoder(embeddings)
        predictions = self.decoder(encoder_outputs, hidden, cell)
        return predictions


## RNN Bidireccional


#### Encoder


In [ ]:
class EncoderBidireccional(nn.Module):
    def __init__(self, embedding_dim=768, hidden_dim=256, num_layers=2, dropout=0.3):
        super(EncoderBidireccional, self).__init__()
        self.lstm = nn.LSTM(
            input_size=embedding_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout,
            bidirectional=True  # bidireccional
        )

    def forward(self, embeddings):
        """
        embeddings: tensor de forma (batch_size, seq_len, embedding_dim)
        """
        outputs, (hidden, cell) = self.lstm(embeddings)
        # outputs: (batch_size, seq_len, hidden_dim)
        # hidden: (num_layers, batch_size, hidden_dim)
        # cell:   (num_layers, batch_size, hidden_dim)
        return outputs, (hidden, cell)

####Decoder

In [ ]:
class DecoderBidireccional(nn.Module):
    def __init__(self, hidden_dim=256, num_layers=2, dropout=0.3):
        super(DecoderBidireccional, self).__init__()
        self.lstm = nn.LSTM(
            input_size=hidden_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout,
            bidirectional=True  # bidireccional
        )

        # Capa feed-forward para cada problema
        self.punt_inicial_ff = nn.Linear(hidden_dim, 2)
        self.punt_final_ff = nn.Linear(hidden_dim, 4)
        self.capital_ff = nn.Linear(hidden_dim, 4)

        # Función de activación para cada problema
        #self.punt_inicial_sigmoid = nn.Sigmoid()
        #self.punt_final_softmax = nn.Softmax(dim=4)
        #self.capital_softmax = nn.Softmax(dim=4)


    def forward(self, encoder_outputs, hidden, cell):
        """
        encoder_outputs: (batch_size, seq_len, hidden_dim)
        hidden, cell: del encoder
        """
        outputs, _ = self.lstm(encoder_outputs, (hidden, cell))

        #punt_inicial_logits = self.punt_inicial_sigmoid(self.punt_inicial_ff(outputs))
        #punt_final_logits = self.punt_final_sofmax(self.punt_final_ff(outputs))
        #capital_logits = self.capital_sofmax(self.capital_ff(outputs))

        punt_inicial_logits = self.punt_inicial_ff(outputs)
        punt_final_logits = self.punt_final_ff(outputs)
        capital_logits = self.capital_ff(outputs)

        return {
            "puntuación inicial": punt_inicial_logits,
            "puntuación final": punt_final_logits,
            "capitalización": capital_logits,
        }


####Encoder-Decoder

In [ ]:
class ModeloBidireccional(nn.Module):
    def __init__(self, embedding_dim=768, hidden_dim=256, num_layers=2, dropout=0.3):
        super(ModeloBidireccional, self).__init__()
        self.encoder = EncoderBidireccional(embedding_dim, hidden_dim, num_layers, dropout)
        self.decoder = DecoderBidireccional(hidden_dim, num_layers, dropout)

    def forward(self, embeddings):
        encoder_outputs, (hidden, cell) = self.encoder(embeddings)
        predictions = self.decoder(encoder_outputs, hidden, cell)
        return predictions


##ENTRENAR MODELO


In [34]:
from sklearn.model_selection import train_test_split
import torch.nn.utils as nn_utils

# Dividir en train/validation
train_df, val_df = train_test_split(df_por_parrafos_test, test_size=0.15, random_state=42)

train_dataset = DynamicEmbeddingDataset(train_df.reset_index(drop=True), tokenizer, model_bert)
val_dataset = DynamicEmbeddingDataset(val_df.reset_index(drop=True), tokenizer, model_bert)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, collate_fn=collate_fn)

model = ModeloUnidireccional(embedding_dim=768, hidden_dim=256, num_layers=2, dropout=0.3)
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

# Mover BERT a la misma device
model_bert = model_bert.to(device)

# Optimizador con weight decay (regularización L2)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=0.01)

# Learning rate scheduler
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.5, patience=3
)

# Pesos diferentes para cada tarea (opcional, ajustar según importancia)
criterion = torch.nn.CrossEntropyLoss(ignore_index=-100)

# Gradient clipping
max_grad_norm = 1.0

num_epochs = 30
best_val_loss = float('inf')
patience_counter = 0
early_stop_patience = 30

for epoch in range(num_epochs):
    # ENTRENAMIENTO
    model.train()
    total_train_loss = 0
    total_loss_inicial = 0
    total_loss_final = 0
    total_loss_cap = 0

    for embeddings, labels in train_loader:
        embeddings = embeddings.to(device)
        labels = {k: v.to(device) for k, v in labels.items()}

        optimizer.zero_grad()
        outputs = model(embeddings)

        loss_inicial = criterion(outputs["puntuación inicial"].permute(0,2,1), labels["punt_inicial"])
        loss_final = criterion(outputs["puntuación final"].permute(0,2,1), labels["punt_final"])
        loss_cap = criterion(outputs["capitalización"].permute(0,2,1), labels["capitalizacion"])

        # Pérdida total (puedes ponderar cada tarea si lo deseas)
        loss = loss_inicial + loss_final + loss_cap

        loss.backward()

        # Gradient clipping para evitar gradientes explosivos
        nn_utils.clip_grad_norm_(model.parameters(), max_grad_norm)

        optimizer.step()

        total_train_loss += loss.item()
        total_loss_inicial += loss_inicial.item()
        total_loss_final += loss_final.item()
        total_loss_cap += loss_cap.item()

    avg_train_loss = total_train_loss / len(train_loader)

    # VALIDACIÓN
    model.eval()
    total_val_loss = 0
    val_loss_inicial = 0
    val_loss_final = 0
    val_loss_cap = 0

    with torch.no_grad():
        for embeddings, labels in val_loader:
            embeddings = embeddings.to(device)
            labels = {k: v.to(device) for k, v in labels.items()}

            outputs = model(embeddings)

            loss_inicial = criterion(outputs["puntuación inicial"].permute(0,2,1), labels["punt_inicial"])

            loss_final = criterion(outputs["puntuación final"].permute(0,2,1), labels["punt_final"])

            loss_cap = criterion(outputs["capitalización"].permute(0,2,1), labels["capitalizacion"])

            loss = loss_inicial + loss_final + loss_cap

            total_val_loss += loss.item()
            val_loss_inicial += loss_inicial.item()
            val_loss_final += loss_final.item()
            val_loss_cap += loss_cap.item()

    avg_val_loss = total_val_loss / len(val_loader)

    # Actualizar learning rate
    scheduler.step(avg_val_loss)

    # Imprimir métricas detalladas
    print(f"Época {epoch+1}/{num_epochs}")
    print(f"  Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")
    print(f"  Train - Inicial: {total_loss_inicial/len(train_loader):.4f}, Final: {total_loss_final/len(train_loader):.4f}, Cap: {total_loss_cap/len(train_loader):.4f}")
    print(f"  Val   - Inicial: {val_loss_inicial/len(val_loader):.4f}, Final: {val_loss_final/len(val_loader):.4f}, Cap: {val_loss_cap/len(val_loader):.4f}")
    print(f"  LR: {optimizer.param_groups[0]['lr']:.6f}")

    # Guardar mejor modelo
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        patience_counter = 0
        ruta_mejor_modelo = "/content/drive/MyDrive/modelos/mejor_modelo.pt"
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_loss': avg_val_loss,
        }, ruta_mejor_modelo)
        print(f"  ✓ Mejor modelo guardado (Val Loss: {avg_val_loss:.4f})")
    else:
        patience_counter += 1
        print(f"  Sin mejora ({patience_counter}/{early_stop_patience})")

    # Early stopping
    if patience_counter >= early_stop_patience:
        print(f"\nEarly stopping en época {epoch+1}")
        break

    print("-" * 60)

# Guardar modelo final
ruta_modelo_final = "/content/drive/MyDrive/modelos/modelo_final_30e.pt"
torch.save({
    'epoch': epoch,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'val_loss': avg_val_loss,
}, ruta_modelo_final)

print(f"\nEntrenamiento completado!")
print(f"Modelo final guardado en: {ruta_modelo_final}")
print(f"Mejor modelo guardado en: {ruta_mejor_modelo}")
print(f"Mejor Val Loss: {best_val_loss:.4f}")


Época 1/30
  Train Loss: 0.7410 | Val Loss: 0.6351
  Train - Inicial: 0.0173, Final: 0.3377, Cap: 0.3860
  Val   - Inicial: 0.0120, Final: 0.3023, Cap: 0.3208
  LR: 0.001000
  ✓ Mejor modelo guardado (Val Loss: 0.6351)
------------------------------------------------------------
Época 2/30
  Train Loss: 0.5497 | Val Loss: 0.4946
  Train - Inicial: 0.0106, Final: 0.2757, Cap: 0.2634
  Val   - Inicial: 0.0103, Final: 0.2583, Cap: 0.2260
  LR: 0.001000
  ✓ Mejor modelo guardado (Val Loss: 0.4946)
------------------------------------------------------------
Época 3/30
  Train Loss: 0.4715 | Val Loss: 0.4458
  Train - Inicial: 0.0090, Final: 0.2553, Cap: 0.2073
  Val   - Inicial: 0.0086, Final: 0.2485, Cap: 0.1887
  LR: 0.001000
  ✓ Mejor modelo guardado (Val Loss: 0.4458)
------------------------------------------------------------
Época 4/30
  Train Loss: 0.4349 | Val Loss: 0.4273
  Train - Inicial: 0.0084, Final: 0.2455, Cap: 0.1810
  Val   - Inicial: 0.0083, Final: 0.2439, Cap: 0.1750
 

In [24]:
def check_df_consistency(dataframe):
    """
    Verifica la consistencia de las longitudes de 'token', 'punt_inicial',
    'punt_final' y 'capitalización' en cada fila del DataFrame.
    """
    mismatched_instances = []
    for idx, row in dataframe.iterrows():
        len_tokens = len(row["token"])
        len_punt_inicial = len(row["punt_inicial"])
        len_punt_final = len(row["punt_final"])
        len_capitalizacion = len(row["capitalización"])

        if not (len_tokens == len_punt_inicial == len_punt_final == len_capitalizacion):
            mismatched_instances.append(row["instancia_id"])
    return mismatched_instances

print("Realizando verificación de consistencia en df_por_parrafos...")
inconsistent_ids = check_df_consistency(df_por_parrafos)

if inconsistent_ids:
    print(f"¡Advertencia! Se encontraron {len(inconsistent_ids)} instancias con longitudes inconsistentes.")
    print(f"Primeras 10 instancia_id con problemas: {inconsistent_ids[:10]}")
    print("Es recomendable filtrar estas instancias antes de proceder con el entrenamiento.")
else:
    print("¡Excelente! Todas las instancias en df_por_parrafos tienen longitudes consistentes.")




Realizando verificación de consistencia en df_por_parrafos...
¡Excelente! Todas las instancias en df_por_parrafos tienen longitudes consistentes.


In [ ]:
"""##INFERENCIA - CARGAR MODELO Y HACER PREDICCIONES"""

def predecir_parrafo(texto, modelo, tokenizer, bert_model, device):
    """
    Recibe un párrafo de texto y devuelve las predicciones de puntuación y capitalización.

    Args:
        texto: string con el texto a procesar
        modelo: modelo entrenado
        tokenizer: tokenizer de BERT
        bert_model: modelo BERT para embeddings
        device: 'cuda' o 'cpu'

    Returns:
        dict con tokens y sus predicciones
    """
    modelo.eval()

    # Tokenizar el texto
    tokens = tokenizer.tokenize(texto)

    # Obtener embeddings
    token_embeddings = []
    for token in tokens:
        token_id = tokenizer.convert_tokens_to_ids(token)
        if token_id is None or token_id == tokenizer.unk_token_id:
            token_id = tokenizer.unk_token_id
        emb = bert_model.embeddings.word_embeddings.weight[token_id].detach()
        token_embeddings.append(emb)

    # Convertir a tensor y añadir dimensión de batch
    embeddings = torch.stack(token_embeddings).unsqueeze(0).to(device)  # (1, seq_len, 768)

    # Hacer predicción
    with torch.no_grad():
        outputs = modelo(embeddings)

    # Obtener las clases predichas
    punt_inicial_pred = torch.argmax(outputs["puntuación inicial"], dim=-1).squeeze().cpu().numpy()
    punt_final_pred = torch.argmax(outputs["puntuación final"], dim=-1).squeeze().cpu().numpy()
    capital_pred = torch.argmax(outputs["capitalización"], dim=-1).squeeze().cpu().numpy()

    # Mapeos de clases a etiquetas legibles
    punt_inicial_map = {0: "Sin puntuación", 1: "Con puntuación"}
    punt_final_map = {0: "Ninguna", 1: "Punto", 2: "Coma", 3: "Otro"}
    capital_map = {0: "Minúscula", 1: "Primera mayúscula", 2: "Todo mayúsculas", 3: "Otro"}

    # Crear resultado
    resultados = []
    for i, token in enumerate(tokens):
        resultados.append({
            "token": token,
            "puntuación_inicial": punt_inicial_map.get(int(punt_inicial_pred[i]), "Desconocido"),
            "puntuación_final": punt_final_map.get(int(punt_final_pred[i]), "Desconocido"),
            "capitalización": capital_map.get(int(capital_pred[i]), "Desconocido")
        })

    return resultados

def reconstruir_texto(resultados):
    """
    Reconstruye el texto con puntuación y capitalización a partir de las predicciones.
    """
    texto_reconstruido = ""

    for resultado in resultados:
        token = resultado["token"]

        # Aplicar capitalización
        if resultado["capitalización"] == "Primera mayúscula":
            token = token.capitalize()
        elif resultado["capitalización"] == "Todo mayúsculas":
            token = token.upper()

        # Añadir puntuación inicial
        if resultado["puntuación_inicial"] == "Con puntuación":
            # Aquí podrías decidir qué puntuación añadir (ej. ¿, ¡, etc.)
            pass

        # Añadir el token
        if token.startswith("##"):
            texto_reconstruido += token[2:]
        else:
            if texto_reconstruido:
                texto_reconstruido += " " + token
            else:
                texto_reconstruido += token

        # Añadir puntuación final
        if resultado["puntuación_final"] == "Punto":
            texto_reconstruido += "."
        elif resultado["puntuación_final"] == "Coma":
            texto_reconstruido += ","

    return texto_reconstruido

In [ ]:

# Cargar el mejor modelo
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Cargando el mejor modelo...")
checkpoint = torch.load("/content/drive/MyDrive/colab/modelos/mejor_modelo.pt", map_location=device)
modelo_cargado = ModeloUnidireccional(embedding_dim=768, hidden_dim=256, num_layers=2, dropout=0.3)
modelo_cargado.load_state_dict(checkpoint['model_state_dict'])
modelo_cargado = modelo_cargado.to(device)
print(f"Modelo cargado (época {checkpoint['epoch']}, val_loss: {checkpoint['val_loss']:.4f})")

# Ejemplo de uso
texto_ejemplo = "el sábado mariana fue al mercado a comprar una sandía luego volvió a su casa y la guardó en la heladera"
print(f"\nTexto de entrada: '{texto_ejemplo}'\n")

predicciones = predecir_parrafo(texto_ejemplo, modelo_cargado, tokenizer, model_bert, device)

# Mostrar predicciones detalladas
print("Predicciones por token:")
print("-" * 80)
for pred in predicciones:
    print(f"Token: {pred['token']:15s} | Punt. Inicial: {pred['puntuación_inicial']:20s} | "
          f"Punt. Final: {pred['puntuación_final']:10s} | Cap: {pred['capitalización']}")

# Reconstruir texto
texto_reconstruido = reconstruir_texto(predicciones)
print("\n" + "=" * 80)
print(f"Texto reconstruido: '{texto_reconstruido}'")
print("=" * 80)

Cargando el mejor modelo...
Modelo cargado (época 14, val_loss: 0.3774)

Texto de entrada: 'el sábado mariana fue al mercado a comprar una sandía luego volvió a su casa y la guardó en la heladera'

Predicciones por token:
--------------------------------------------------------------------------------
Token: el              | Punt. Inicial: Sin puntuación       | Punt. Final: Ninguna    | Cap: Primera mayúscula
Token: s               | Punt. Inicial: Sin puntuación       | Punt. Final: Ninguna    | Cap: Primera mayúscula
Token: ##ába           | Punt. Inicial: Sin puntuación       | Punt. Final: Ninguna    | Cap: Primera mayúscula
Token: ##do            | Punt. Inicial: Sin puntuación       | Punt. Final: Ninguna    | Cap: Minúscula
Token: mari            | Punt. Inicial: Sin puntuación       | Punt. Final: Ninguna    | Cap: Minúscula
Token: ##ana           | Punt. Inicial: Sin puntuación       | Punt. Final: Ninguna    | Cap: Minúscula
Token: fue             | Punt. Inicial: Sin puntu